In [1]:
from selenium import webdriver
import requests
import pandas as pd
import os
import platform
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementClickInterceptedException, NoSuchElementException, TimeoutException, ElementNotInteractableException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

import json
from requests import Session
import time
from json import JSONDecodeError

In [2]:
def get_webdriver_path():
    
    my_os = platform.system()
    
    if my_os == 'Windows':
        return os.path.abspath(os.path.join('gecko_drivers', 'win64.exe'))
    else:
        return os.path.abspath(os.path.join('gecko_drivers', 'linux64'))

In [3]:
def listar_links_temas(home):
    
    with requests.get(home) as r:
        html = r.text
    
    sopa = BeautifulSoup(html)
    box_icones = sopa.find('div', {'id' : "home-box-indicadores"})
    icones = box_icones.find_all('a', {'class' : "link-padrao d-block text-center"})
    
    if home.endswith(r'/'):
        home = home[:-1]
    
    links = []
    for icone in icones:
        slug = icone.get('href')
        link = home + slug
        links.append(link)
        
    return links

In [4]:
def entrar_pagina_tema(browser, link_tema):
    
    browser.get(link_tema)

def listar_elementos_tema(browser):
    
    els = browser.find_elements_by_class_name('item-indicador-search')
    
    return els

def parsear_elemento_tema(el):
    
    desc_indi = el.get_attribute('title')
    cod_indi = el.get_attribute('cd_indicador')
    
    parsed = {
            'codigo' : cod_indi,
            'desc' : desc_indi,
            'element' : el
        }
    
    return parsed

def parsear_todos_els_tema(els):
    
    parsed_data = {}
    
    for el in els:
        data = parsear_elemento_tema(el)
        codigo = data['codigo']
        parsed_data[codigo] = data
    
    return parsed_data
        

In [5]:
def wait_aparecer_animacao(browser):


    try:
        element = WebDriverWait(browser, 60).until(
            EC.visibility_of_element_located((By.CLASS_NAME, "ring"))
        )
    except Exception as e:
        print(e)

def wait_desaparecer_animacao(browser):


    try:
        element = WebDriverWait(browser, 60).until(
            EC.invisibility_of_element_located((By.CLASS_NAME, "ring"))
        )
    except Exception as e:
        print(e)
              
              
def clickar_animacao(browser, element):
    
    try:
        element.click()
    except ElementClickInterceptedException:
        browser.execute_script("arguments[0].scrollIntoView();", element)

    #primeira wait para garantir que animacao ainda nao apareceu
    wait_desaparecer_animacao(browser)
    
    wait_aparecer_animacao(browser)
    
    wait_desaparecer_animacao(browser)


In [6]:
def wait_card_periodos(browser):
    
    element = WebDriverWait(browser, 30).until(
            EC.visibility_of_element_located((By.CLASS_NAME, "item-periodo-search"))
        )

def wait_periodo(browser, el_periodo):
    
    element = WebDriverWait(browser, 30).until(
                EC.element_to_be_clickable(el_periodo)
            )

def listar_periodos_indicador(browser):
    
    wait_card_periodos(browser)
    
    primeiro_periodo = browser.find_element_by_class_name('item-periodo-search')
    
    wait_periodo(browser, primeiro_periodo)

    periodos = browser.find_elements_by_class_name('item-periodo-search')
    
    return periodos

def back_to_top(browser):
    
    browser.switch_to.default_content()
    body = browser.find_element_by_tag_name('body')
    body.send_keys(Keys.CONTROL + Keys.HOME)
    
def back_top_reg(browser):
    
    back_to_top(browser)
    
    el = browser.find_element_by_id("badge-localidade")
    
    browser.execute_script("arguments[0].scrollIntoView();", el)


def clicar_periodos(browser, periodos):
    
    for el in periodos:
        try:
            
            wait_periodo(browser, el)
            
            el.click()

        except ElementClickInterceptedException:

            #scroll down para aparecer o item
            browser.execute_script("arguments[0].scrollIntoView();", el)

            wait_periodo(browser, el)

            el.click()
            
    back_top_reg(browser)

In [7]:
def get_arvore_reg(browser):
    
    letras = ('x', 'y')
    
    for letra in letras:
        try:
            css_selector = f'li[aria-describedby="shieldd{letra}"]'
            arvore_reg = browser.find_element_by_css_selector(css_selector)
            return arvore_reg
        except NoSuchElementException:
            pass
    
    return browser.find_element_by_id('shielddw')


def clicar_regionalizacao(browser):
    
    back_top_reg(browser)

    arvore_reg = get_arvore_reg(browser)
    browser.execute_script("arguments[0].scrollIntoView();", arvore_reg)

    WebDriverWait(browser, 30).until(
                EC.element_to_be_clickable(arvore_reg)
            )
    #dois cliques abre o menu
    actionChains = ActionChains(browser)
    actionChains.double_click(arvore_reg).perform()
        
    #um clique seleciona geral
    arvore_reg.click()

In [8]:
def pesquisar(browser):
    
    botao = browser.find_element_by_id('btn-search')
    
    botao.click()
    
    wait_desaparecer_animacao(browser)
    
    wait_aparecer_animacao(browser)
    
    wait_desaparecer_animacao(browser)

In [9]:
def download_json(browser):
    
    session = Session()
    
    selenium_cookies = browser.get_cookies()
    for cookie in selenium_cookies:
        session.cookies.set(cookie["name"], cookie["value"])
        
    link_download = 'https://observasampa.prefeitura.sp.gov.br/PesquisaDeIndicadores/DownloadIndicador/.json/Indicadores'
    
    with session.get(link_download) as r:
        c = r.content
        t = c.decode('latin-1')
        t = t.encode('utf-8')
        try:
            dados = json.loads(t)
        except JSONDecodeError:
            dados = [t.decode('utf-8')]
        
    return dados

In [10]:
def check_botao_sem_dados(browser):
    
    try:
        css_selector = 'button[data-dismiss="modal"]'
        botao_dismiss = browser.find_element_by_css_selector(css_selector)
        if botao_dismiss.is_enabled():
            modal = browser.switch_to.active_element
            modal.click()
            
            return True
    except (NoSuchElementException, ElementNotInteractableException):
        pass
    

In [11]:
def loop_tema(browser, link_tema, dados_retorno):
    
    entrar_pagina_tema(browser, link_tema)
    els = listar_elementos_tema(browser)
    els = parsear_todos_els_tema(els)
    
    for codigo, el in els.items():
        try:
            not_stale = listar_elementos_tema(browser)
            not_stale = parsear_todos_els_tema(not_stale)

            elemento = not_stale[codigo]['element']

            el.pop('element')

            clickar_animacao(browser, elemento)
            
            check_botao_sem_dados(browser)

            periodos = listar_periodos_indicador(browser)
            clicar_periodos(browser, periodos)

            clicar_regionalizacao(browser)

            pesquisar(browser)

            dados = download_json(browser)

            el['data'] = dados
            dados_retorno[codigo] = el
        
        except TimeoutException:
            
            el['data'] = []
            el['error'] = True
            dados_retorno[codigo] = el
        
        entrar_pagina_tema(browser, link_tema)
        check_botao_sem_dados(browser)
        
    return els

In [12]:
if not os.path.exists('generated_data'):
    os.mkdir('generated_data')

In [13]:
browser = webdriver.Firefox(executable_path=get_webdriver_path())

<ipython-input-13-62dd6c1cd0ec>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Firefox(executable_path=get_webdriver_path())


In [14]:
home = 'https://observasampa.prefeitura.sp.gov.br/'

In [15]:
browser.get(home)

In [16]:
links_temas = listar_links_temas(home)

In [17]:
for i, tema in enumerate(links_temas):
    
    
    nom_file = tema.split('/')[-1] + '.json'
    print(nom_file)
    print((i/len(links_temas))*100)
    path_file = os.path.join('generated_data', nom_file)
    if os.path.exists(path_file):
        print(f'{nom_file} já parseado')
        continue
        
    dados = {}
    
    loop_tema(browser, tema, dados)
    
    
    
    with open(path_file, 'w') as f:
        json.dump(dados, f)

financas-publicas.json
0.0
financas-publicas.json já parseado
educacao.json
5.263157894736842
educacao.json já parseado
esporte-e-lazer.json
10.526315789473683
esporte-e-lazer.json já parseado
gestao-publica.json
15.789473684210526
gestao-publica.json já parseado
meio-ambiente.json
21.052631578947366
meio-ambiente.json já parseado
mobilidade-e-seguranca-no-transito.json
26.31578947368421
mobilidade-e-seguranca-no-transito.json já parseado
moradia-e-saneamento-basico.json
31.57894736842105
moradia-e-saneamento-basico.json já parseado
populacao.json
36.84210526315789
populacao.json já parseado
economia-trabalho-e-renda.json
42.10526315789473
economia-trabalho-e-renda.json já parseado
saude.json
47.368421052631575
saude.json já parseado
seguranca-e-violencia.json
52.63157894736842
seguranca-e-violencia.json já parseado
turismo.json
57.89473684210527
turismo.json já parseado
cultura.json
63.1578947368421
cultura.json já parseado
acessibilidade-e-pessoa-com-deficiencia.json
68.4210526315789

In [18]:
browser.close()